<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Setup

In [1]:
!pip install hopsworks

  Using cached hopsworks-3.4.4-py3-none-any.whl
  Using cached hsfs-3.4.7-py3-none-any.whl
  Using cached hsml-3.4.6-py3-none-any.whl
  Using cached pyhumps-1.6.1-py3-none-any.whl (5.0 kB)
  Using cached furl-2.1.3-py2.py3-none-any.whl (20 kB)
  Using cached pyjks-20.0.0-py2.py3-none-any.whl (45 kB)
  Using cached avro-1.11.0-py2.py3-none-any.whl
  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached great_expectations-0.14.13-py3-none-any.whl (5.0 MB)
  Using cached MarkupSafe-2.0.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (30 kB)
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
  Using cached altair-4.2.2-py3-none-any.whl.metadata (13 kB)
  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached ruamel.yaml-0.17.17-py3-none-any.whl (109 kB)
  Using cached PyHopsHive-0.6.4.1.dev0-py3-none-any.whl
  Using cached confluent_k

In [2]:
!pip install tensorflow

  Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 3.0.1 requires Jinja2>=3.1.2, but you have jinja2 3.0.3 which is incompatible.
hsfs 3.4.7 requires markupsafe<2.1.0, but you have markupsafe 2.1.5 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.


In [18]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 19.5 MB/s eta 0:00:00


<div style="background-color:teal; color:white; padding:5px; font-size:20px">
Imports

In [3]:
import tensorflow as tf
import pandas as pd
import pprint
import numpy as np

import importlib
import sys
sys.path.append('./scripts')

2024-02-20 19:14:24.119290: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
import boto3

# Initialize a client for Systems Manager
ssm = boto3.client('ssm', region_name='us-east-1')

parameter_name = 'hopsworks-api-key'

# Fetch the parameter
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)

# Extract the parameter value (API key in this case)
hopsworks_api_key = response['Parameter']['Value']

WxdCPfqMaXQDwa5I.3OUkayjn6M2ogFG0g0MByyBgfdmHZZ0eWm5DrDkEhcWZOiD4CI5VkNvd48oCGG8v


In [22]:
# Pinecone Secrets
parameter_name = 'pinecone-recsys-api-key'
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)  # Fetch parameter
pinecone_recsys_api_key = response['Parameter']['Value']  # Extract parameter value

<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Hopsworks Feature Store Connection

In [5]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/475285
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Model & Data Imports

In [6]:
# Fetch Model From Registry
model = mr.get_model(
    name="candidate_model",
    version=1,
)

model_path = model.download()  # Set download path
candidate_model = tf.saved_model.load(model_path)  # Load Model

In [7]:
# Download Training Features 
feature_view = fs.get_feature_view(name="retrieval", version=1)

train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)

Finished: Reading data from Hopsworks, using ArrowFlight (111.81s) 


In [8]:
train_df.head(3)

,customer_id,article_id,t_dat,price,month_sin,month_cos,age,club_member_status,age_group,garment_group_name,index_group_name
0,e2eb4cae50256f162513c51f2ac209dcaed12dab7cf168...,510465001,1538092800000,0.025407,-1.0,-1.836970e-16,36.0,ACTIVE,36-45,Jersey Fancy,Ladieswear
1,2ec4adb30e3ee3e9916f0e50acec4993c8a416266dede5...,568601013,1538092800000,0.025407,-1.0,-1.836970e-16,48.0,ACTIVE,46-55,Dressed,Ladieswear
3,1a89521d786a5d2d96a6de983961e998f1d8a2f980875e...,667491002,1537488000000,0.025407,-1.0,-1.836970e-16,23.0,ACTIVE,19-25,Blouses,Ladieswear


In [9]:
# Set cat. variables as strings
train_df["article_id"] = train_df["article_id"].astype(str)
val_df["article_id"] = val_df["article_id"].astype(str)

<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Create Candidate Item Embeddings

In [10]:
# Get list of input features for the candidate model.
model_schema = model.model_schema['input_schema']['columnar_schema']
candidate_features = [feat['name'] for feat in model_schema]
candidate_features

['article_id', 'garment_group_name', 'index_group_name']

In [11]:
# Get list of unique candidate items.
item_df = train_df[candidate_features]
item_df.drop_duplicates(subset="article_id", inplace=True)

# Convert to TF Dataset
item_ds = tf.data.Dataset.from_tensor_slices(
    {col: item_df[col] for col in item_df})

# Compute embeddings for all candidate items.
candidate_embeddings = item_ds.batch(2048).map(
    lambda x: (x["article_id"], candidate_model(x)))

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Prep Data For Upserting

In [35]:
# Concatenate all article IDs and embeddings from the candidate_embeddings dataset
all_article_ids = tf.concat([batch[0] for batch in candidate_embeddings], axis=0)
all_embeddings = tf.concat([batch[1] for batch in candidate_embeddings], axis=0)

# Convert tensors to numpy arrays
all_article_ids_np = all_article_ids.numpy().astype(int)
all_embeddings_np = all_embeddings.numpy()

# Convert numpy arrays to lists
items_ids_list = all_article_ids_np.tolist()
embeddings_list = all_embeddings_np.tolist()

# Create a list of dictionaries for upserting
to_upsert = [{"id": str(item_id), "values": embedding} for item_id, embedding in zip(items_ids_list, embeddings_list)]

<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Upsert Into Pinecone Vector Database

In [36]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_recsys_api_key)
index = pc.Index("recsys-project")

In [37]:
pc.list_indexes()

{'indexes': [{'dimension': 16,
              'host': 'recsys-project-oyplaws.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'recsys-project',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [39]:
import itertools
from pinecone import Pinecone


def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [40]:
# Upsert data with 100 vectors per upsert request
for i in chunks(to_upsert, batch_size=100):
    index.upsert(vectors=i) 

In [44]:
# check number of records in the index
index.describe_index_stats()

{'dimension': 16,
 'index_fullness': 0.24569,
 'namespaces': {'': {'vector_count': 24569}},
 'total_vector_count': 24569}

<div style="background-color:moccasin; color:black; padding:5px; font-size:20px">
Upload Embeddings to Hopsworks Feature Store

In [45]:
# Create a DataFrame
data_emb = pd.DataFrame({
    'article_id': items_ids_list, 
    'embeddings': embeddings_list,
})

data_emb.head()

,article_id,embeddings
0,510465001,"[2.3589532375335693, -3.0936050415039062, -2.2..."
1,568601013,"[0.45300203561782837, -1.622733235359192, -3.4..."
2,667491002,"[0.16190499067306519, -2.651552438735962, -2.1..."
3,662857001,"[-0.23621296882629395, -1.194600224494934, -2...."
4,651285001,"[0.9611461162567139, -1.1432050466537476, -2.2..."


In [46]:
# Create'candidate_embeddings_fg' feature group
candidate_embeddings_fg = fs.get_or_create_feature_group(
    name="candidate_embeddings_fg",
    primary_key=['article_id'],
    version=1,
    description='Embeddings for each article',
    online_enabled=True,
)

candidate_embeddings_fg.insert(data_emb)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/475285/fs/471108/fg/543080


Uploading Dataframe: 0.00% |          | Rows 0/24569 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: candidate_embeddings_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/475285/jobs/named/candidate_embeddings_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f5d803bd540>, None)

In [47]:
# Create 'candidate_embeddings' feature view
feature_view = fs.get_or_create_feature_view(
    name="candidate_embeddings",
    version=1,
    description='Embeddings of each article',
    query=candidate_embeddings_fg.select(["article_id"]),
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/475285/fs/471108/fv/candidate_embeddings/version/1
